# Pre-processing_final
This notebook aggregates 3 preprocessed datasets

In [1]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sbs
import geopandas as gpd
import folium 


In [2]:
from pyspark.sql import SparkSession

# Create a spark session with increased memory allocation
spark = (
    SparkSession.builder.appName("ADS Project1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "8g")  # Set the driver memory to 8GB
    .config("spark.executor.memory", "8g")  # Set the executor memory to 8GB
    .getOrCreate()
)

24/08/27 18:47:44 WARN Utils: Your hostname, Hanshis-Laptop.local resolves to a loopback address: 127.0.0.1; using 10.12.200.32 instead (on interface en0)
24/08/27 18:47:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/27 18:47:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/27 18:47:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
spark.conf.set("spark.sql.parquet.compression.codec","gzip")

## Datasets 

In [5]:
# read the data from preprocessed tlc data
tdf = spark.read.parquet("../data/raw/tlc_df.parquet") 

In [8]:
# read the preprocessed weather data
wdf = spark.read.csv("../data/raw/NYC_weather_raw.csv", header=True, inferSchema=True)


In [9]:
# read the preprocessed event data
edf = spark.read.parquet("../data/raw/NYC_Permitted_Event_Information_Historical.parquet") 

In [10]:
# show 5 rows of the tlc data
tdf.show(5)

+--------+-----------+-----------+------------+------------+-------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+--------------------+---------+------------+------------+
|VendorID|pickup_date|pickup_hour|dropoff_date|dropoff_hour|trip_duration|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|congestion_surcharge|ehail_fee|total_amount|payment_type|
+--------+-----------+-----------+------------+------------+-------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+--------------------+---------+------------+------------+
|       2| 2023-12-01|          0|  2023-12-01|           0|         10.0|             false|         1|          68|          50|

In [11]:
# show 5 rows of the weather data
wdf.show(5)

+----------+----+-------+---+------+---+----+------+
|      DATE|HOUR|    CIG|WND|   VIS|TMP| DEW|   SLP|
+----------+----+-------+---+------+---+----+------+
|2023-12-01|   1|22000.0|4.1|1609.3|9.4|-2.8|1020.1|
|2023-12-01|   2|22000.0|2.1|1609.3|8.9|-2.2|1020.0|
|2023-12-01|   3|22000.0|3.1|1609.3|8.9|-2.2|1020.4|
|2023-12-01|   4|22000.0|3.1|1609.3|8.3|-1.7|1020.7|
|2023-12-01|   5|22000.0|3.1|1609.3|7.8|-1.7|1020.8|
+----------+----+-------+---+------+---+----+------+
only showing top 5 rows



In [12]:
# show 5 rows of the event data
edf.show(5)

+--------+----------+----------+----------+--------+-------------+-------------+--------------------+
|Event ID|Start Date|Start Hour|  End Date|End Hour|   Event Type|Event Borough|      Event Location|
+--------+----------+----------+----------+--------+-------------+-------------+--------------------+
|  684438|2023-12-09|         7|2023-12-09|      10|Special Event|     Brooklyn|Prospect Park: Pi...|
|  693693|2023-12-03|        12|2023-12-03|      14|Special Event|    Manhattan|Central Park: Wag...|
|  686564|2023-12-03|        17|2023-12-03|      18|Special Event|    Manhattan|Carl Schurz Park:...|
|  684416|2023-12-09|         9|2023-12-09|      11|Special Event|    Manhattan|Washington Square...|
|  687023|2023-12-01|        11|2023-12-01|      12|Special Event|    Manhattan|Central Park: Lad...|
+--------+----------+----------+----------+--------+-------------+-------------+--------------------+
only showing top 5 rows



## Standardization

## Feature selection